In [51]:
# Import data and inspect 
h<-read.csv("NoCategoricos5.csv",sep=";")
summary(h)

# It is observed that there 1000 values with 5 non-categorical variables and a target of 5 possible outcomes (0,1,2,3,4)


       X                A                  B                  C            
 Min.   :   1.0   Min.   :0.001312   Min.   :0.004877   Min.   :0.0003158  
 1st Qu.: 250.8   1st Qu.:0.262453   1st Qu.:0.242199   1st Qu.:0.2201134  
 Median : 500.5   Median :0.500340   Median :0.482907   Median :0.4776410  
 Mean   : 500.5   Mean   :0.501791   Mean   :0.490734   Mean   :0.4871791  
 3rd Qu.: 750.2   3rd Qu.:0.741428   3rd Qu.:0.750363   3rd Qu.:0.7600665  
 Max.   :1000.0   Max.   :0.998609   Max.   :0.998830   Max.   :0.9965820  
       D                   E                  R        
 Min.   :0.0009639   Min.   :0.000282   Min.   :0.000  
 1st Qu.:0.2344054   1st Qu.:0.262639   1st Qu.:1.000  
 Median :0.5001730   Median :0.509958   Median :1.000  
 Mean   :0.4978276   Mean   :0.513217   Mean   :1.363  
 3rd Qu.:0.7582356   3rd Qu.:0.767861   3rd Qu.:2.000  
 Max.   :0.9999384   Max.   :0.999214   Max.   :4.000  

In [52]:
# Create an empty vector of random numbers of the same length of the dataset
x <- runif(nrow(h))

In [53]:
# Split data to create 3 random trees using the bootstrap technique

h1 <- h[which(x<1/3.),]
h2 <- h[which(x>=1/3. & x<2/3.),]
h3 <- h[which(x>=2/3.) ,]

hTrainAB <- rbind(h1,h2)
hTestAB <- h3

hTrainAC <- rbind(h1,h3)
hTestAC <- h2

hTrainBC <- rbind(h2,h3)
hTestBC <- h1

In [54]:
# Create different trees and predict

library(rpart)

tAB <- rpart(data = hTrainAB, formula = R ~ A + B + C + D + E)
tBC <- rpart(data = hTrainBC, formula = R ~ A + B + C + D + E)
tAC <- rpart(data = hTrainAC, formula = R ~ A + B + C + D + E)

pAB <- predict(tAB, newdata = hTestAB)
pBC <- predict(tBC, newdata = hTestBC)
pAC <- predict(tAC, newdata = hTestAC)

In [55]:
# Calculate each error with the respective dataset
errAB <- sum((as.vector(pAB)-hTestAB$R)^2)/nrow(hTestAB)
errBC <- sum((as.vector(pBC)-hTestBC$R)^2)/nrow(hTestBC)
errAC <- sum((as.vector(pAC)-hTestAC$R)^2)/nrow(hTestAC)

In [56]:
# Calculate the overall error using the whole dataset (h)
p1 <- as.vector(predict(tAB, newdata = h))
p2 <- as.vector(predict(tBC, newdata = h))
p3 <- as.vector(predict(tAC, newdata = h))

p <- (p1/errAB+p2/errBC+p3/errAC)/(1/errAB+1/errBC+1/errAC)

p <- floor(p+.5)

length(which(p==h$R))/nrow(h)

# 76%

[1] 0.737

In [57]:
# Calculte each error by itself
pAB <- floor(pAB+.5)
pBC <- floor(pBC+.5)
pAC <- floor(pAC+.5)

length(which(pAB==hTestAB$R))/nrow(hTestAB)
length(which(pBC==hTestBC$R))/nrow(hTestBC)
length(which(pAC==hTestAC$R))/nrow(hTestAC)

# Between 63 and 70% of successful predictions

[1] 0.6610169

[1] 0.6424419

[1] 0.6589404

In [58]:
# Using bagging method included in the ipred library

library(ipred)
mb <- bagging(R ~ A + B + C + D + E, data=h, coob=TRUE)
pb <- predict(mb,newdata=h)
pb <- floor(.5+pb)
length(which(pb==h$R))/nrow(h) 

# 78.1% 

[1] 0.775

In [59]:
# Trying with Random Forest

library(randomForest)

rf <- randomForest(formula = R ~ A + B + C + D + E, data = h)

prf <- predict(rf, newdata=h)
prf <- floor(0.5+prf)

length(which(prf == h$R))/nrow(h)

## 95%, best model of the series

Warning message in randomForest.default(m, y, ...):
"The response has five or fewer unique values.  Are you sure you want to do regression?"

[1] 0.948